In [ ]:
import torch
print(torch.version.cuda)

11.6


In [ ]:
import torch 
print(torch.__version__)

1.13.0+cu116


In [ ]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     |████████████████████████████████| 1.9 MB 33.1 MB/s 
     |████████████████████████████████| 9.4 MB 54.7 MB/s 
     |████████████████████████████████| 4.6 MB 67.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 564 kB 28.0 MB/s 
     |████████████████████████████████| 280 kB 78.6 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=1bf1a9906a5e47e9b5ba8de2b70cb0d9e96f157a6af26e9ba12820ee1abcfec0
  Stored in directory: /root/.cache/pip/wheels/59/a3/20/198928106d3169865ae73afcbd3d3d1796cf6b429b55c65378
Successfully built torch-geometric
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninsta

In [2]:
!pip install geopandas
!pip install 'networkx<2.7'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 11.7 MB/s 
     |████████████████████████████████| 7.8 MB 48.3 MB/s 
     |████████████████████████████████| 16.6 MB 42.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 28.9 MB/s 
  Attempting uninstall: networkx
    Found existing installation: networkx 2.8.8
    Uninstalling networkx-2.8.8:
      Successfully uninstalled networkx-2.8.8


In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau, CyclicLR
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
from torch.nn.functional import leaky_relu
import geopandas as gpd

import random
import time

## Data Input

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Preprocessing

## Baseline Models

In [3]:
data_fil = pd.read_csv('/content/drive/MyDrive/DL/data_fil.csv')
data_fil

,Unnamed: 0,SHAPE_Leng,StreetWidt,Number_Tra,Number_Par,Number_Tot,With_BikeLane,Protected_Lane,Unprotected_Lane,Truck,...,Race_White,BachelorHigher,Unemployment,Mode_Public,NoVehicle,Sub_dis,Bike_dis,origin,destination,Flow_Median
0,40330,262.777812,0.4625,0.166667,0.50,0.571429,0,0,0,0,...,0.778634,0.934315,0.022388,0.524324,0.529730,0.238525,0.245659,1601,1606,3.0
1,40331,241.736699,0.4125,0.000000,0.50,0.428571,0,0,0,0,...,0.778634,0.934315,0.022388,0.524324,0.529730,0.271180,0.056131,1608,1598,151.0
2,40332,293.100274,0.3625,0.000000,0.50,0.428571,0,0,0,0,...,0.778634,0.934315,0.022388,0.524324,0.529730,0.198650,0.048528,1614,1601,404.0
3,40333,48.107140,0.5125,0.166667,0.50,0.571429,0,0,0,0,...,0.666300,0.944507,0.020256,0.531892,0.726126,0.113288,0.023830,1778,1777,170.0
4,40334,303.557174,0.5625,0.500000,0.00,0.571429,0,0,0,1,...,0.666300,0.944507,0.020256,0.531892,0.726126,0.013627,0.129543,1684,1691,667.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19779,139981,170.799256,0.3375,0.166667,0.50,0.571429,0,0,0,0,...,0.676762,0.704417,0.063433,0.608108,0.800901,0.456174,0.204919,3216,3264,155.0
19780,139982,43.140752,0.3375,0.166667,0.50,0.571429,0,0,0,0,...,0.632159,0.869762,0.004264,0.504865,0.742342,0.501163,0.272736,3207,3216,155.0
19781,139983,31.293044,0.4375,0.166667,0.00,0.285714,0,0,0,0,...,0.876652,0.869762,0.060768,0.541622,0.668468,0.050450,0.181640,4422,4443,22.0
19782,139984,56.997336,0.5625,0.333333,0.25,0.571429,1,0,1,1,...,0.845815,0.891846,0.052239,0.567568,0.531532,0.036379,0.181640,4444,4422,388.0


In [4]:
bins=[data_fil['Flow_Median'].quantile(0)-1
      , data_fil['Flow_Median'].quantile(0.2)
      , data_fil['Flow_Median'].quantile(0.4)
      , data_fil['Flow_Median'].quantile(0.6)
      , data_fil['Flow_Median'].quantile(0.8)
      ,data_fil['Flow_Median'].quantile(1)+1]

data_fil['Class'] = pd.cut(data_fil['Flow_Median'], bins=bins, labels=[1,2,3,4,5])
data_fil.head()

,Unnamed: 0,SHAPE_Leng,StreetWidt,Number_Tra,Number_Par,Number_Tot,With_BikeLane,Protected_Lane,Unprotected_Lane,Truck,...,BachelorHigher,Unemployment,Mode_Public,NoVehicle,Sub_dis,Bike_dis,origin,destination,Flow_Median,Class
0,40330,262.777812,0.4625,0.166667,0.5,0.571429,0,0,0,0,...,0.934315,0.022388,0.524324,0.529730,0.238525,0.245659,1601,1606,3.0,1
1,40331,241.736699,0.4125,0.000000,0.5,0.428571,0,0,0,0,...,0.934315,0.022388,0.524324,0.529730,0.271180,0.056131,1608,1598,151.0,3
2,40332,293.100274,0.3625,0.000000,0.5,0.428571,0,0,0,0,...,0.934315,0.022388,0.524324,0.529730,0.198650,0.048528,1614,1601,404.0,4
3,40333,48.107140,0.5125,0.166667,0.5,0.571429,0,0,0,0,...,0.944507,0.020256,0.531892,0.726126,0.113288,0.023830,1778,1777,170.0,3
4,40334,303.557174,0.5625,0.500000,0.0,0.571429,0,0,0,1,...,0.944507,0.020256,0.531892,0.726126,0.013627,0.129543,1684,1691,667.0,5


In [5]:
import networkx as nx
import scipy

In [6]:
# building graph for baseline models
def load_data():

    G = nx.from_pandas_edgelist(data_fil, 'origin', 'destination', 'SHAPE_Leng', create_using = nx.DiGraph())
    adj_list = np.array([nx.adjacency_matrix(G).todense()], dtype=float)
    
    init_feat1 = np.array(data_fil.groupby('destination')['StreetWidt'].agg(['unique']))
    init_feat1 = np.array(list(map(lambda x: x[0][0], init_feat1))).reshape(-1, 1)
    init_feat2 = np.array(data_fil.groupby('destination')['Number_Tra'].agg(['unique']))
    init_feat2 = np.array(list(map(lambda x: x[0][0], init_feat2))).reshape(-1, 1)
    init_feat3 = np.array(data_fil.groupby('destination')['Number_Par'].agg(['unique']))
    init_feat3 = np.array(list(map(lambda x: x[0][0], init_feat3))).reshape(-1, 1)
    init_feat4 = np.array(data_fil.groupby('destination')['Number_Tot'].agg(['unique']))
    init_feat4 = np.array(list(map(lambda x: x[0][0], init_feat4))).reshape(-1, 1)
    init_feat5 = np.array(data_fil.groupby('destination')['With_BikeLane'].agg(['unique']))
    init_feat5 = np.array(list(map(lambda x: x[0][0], init_feat5))).reshape(-1, 1)
    init_feat6 = np.array(data_fil.groupby('destination')['Protected_Lane'].agg(['unique']))
    init_feat6 = np.array(list(map(lambda x: x[0][0], init_feat6))).reshape(-1, 1)
    init_feat7 = np.array(data_fil.groupby('destination')['Unprotected_Lane'].agg(['unique']))
    init_feat7 = np.array(list(map(lambda x: x[0][0], init_feat7))).reshape(-1, 1)
    init_feat8 = np.array(data_fil.groupby('destination')['Truck'].agg(['unique']))
    init_feat8 = np.array(list(map(lambda x: x[0][0], init_feat8))).reshape(-1, 1)
    #init_feat9 = np.array(data_fil.groupby('destination')['Manhattan'].agg(['unique']))
    #init_feat9 = np.array(list(map(lambda x: x[0][0], init_feat9))).reshape(-1, 1)
    #init_feat10 = np.array(data_fil.groupby('destination')['Bronx'].agg(['unique']))
    #init_feat10 = np.array(list(map(lambda x: x[0][0], init_feat10))).reshape(-1, 1) 
    #init_feat11 = np.array(data_fil.groupby('destination')['Brooklyn'].agg(['unique']))
    #init_feat11 = np.array(list(map(lambda x: x[0][0], init_feat11))).reshape(-1, 1)
    #init_feat12 = np.array(data_fil.groupby('destination')['Queens'].agg(['unique']))
    #init_feat12 = np.array(list(map(lambda x: x[0][0], init_feat12))).reshape(-1, 1)
    init_feat13 = np.array(data_fil.groupby('destination')['Pop'].agg(['unique']))
    init_feat13 = np.array(list(map(lambda x: x[0][0], init_feat13))).reshape(-1, 1)
    init_feat14 = np.array(data_fil.groupby('destination')['Median_Age'].agg(['unique']))
    init_feat14 = np.array(list(map(lambda x: x[0][0], init_feat14))).reshape(-1, 1)
    init_feat15 = np.array(data_fil.groupby('destination')['Median_Income'].agg(['unique']))
    init_feat15 = np.array(list(map(lambda x: x[0][0], init_feat15))).reshape(-1, 1)
    init_feat16 = np.array(data_fil.groupby('destination')['Race_White'].agg(['unique']))
    init_feat16 = np.array(list(map(lambda x: x[0][0], init_feat16))).reshape(-1, 1)
    init_feat17 = np.array(data_fil.groupby('destination')['BachelorHigher'].agg(['unique']))
    init_feat17 = np.array(list(map(lambda x: x[0][0], init_feat17))).reshape(-1, 1)
    init_feat18 = np.array(data_fil.groupby('destination')['Unemployment'].agg(['unique']))
    init_feat18 = np.array(list(map(lambda x: x[0][0], init_feat18))).reshape(-1, 1)
    init_feat19 = np.array(data_fil.groupby('destination')['Mode_Public'].agg(['unique']))
    init_feat19 = np.array(list(map(lambda x: x[0][0], init_feat19))).reshape(-1, 1)
    init_feat20 = np.array(data_fil.groupby('destination')['NoVehicle'].agg(['unique']))
    init_feat20 = np.array(list(map(lambda x: x[0][0], init_feat20))).reshape(-1, 1)
    init_feat21 = np.array(data_fil.groupby('destination')['Sub_dis'].agg(['unique']))
    init_feat21 = np.array(list(map(lambda x: x[0][0], init_feat21))).reshape(-1, 1)
    init_feat22 = np.array(data_fil.groupby('destination')['Bike_dis'].agg(['unique']))
    init_feat22 = np.array(list(map(lambda x: x[0][0], init_feat22))).reshape(-1, 1)
    


    init_feat = np.concatenate([init_feat1, init_feat2, init_feat3, init_feat4, init_feat5, init_feat6, init_feat7, init_feat8, #init_feat9, init_feat10, init_feat11, init_feat12, 
                                init_feat13, init_feat14, init_feat15, init_feat16, init_feat17, init_feat18, init_feat19, init_feat20, init_feat21, init_feat22], axis=1)
    
    true_label = np.array(data_fil.groupby('destination')['Class'].agg(['unique']))
    true_label = np.array(list(map(lambda x: x[0][0], true_label))).reshape(-1, 1)

    return adj_list, init_feat, true_label

adj, feature, labels = load_data()

features = np.expand_dims(feature, axis=0)

In [7]:
# Splitting the features and label into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(feature, labels, test_size = 0.30, random_state = 433)

In [8]:
#Importing different classifier from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
# Define scoring method
scoring = 'accuracy'

# Baseline Model building to train
names = [ 'Decision Tree', 'Random Forest', 'Neural Net',
         'AddaBoost', 'Naive Bayes', 'SVM Linear', 'SVM Sigmoid']
Classifiers = [
    DecisionTreeClassifier(max_depth = 5),
    RandomForestClassifier(max_depth = 5, n_estimators = 10, max_features = 1 ),
    MLPClassifier(alpha = 1),
    AdaBoostClassifier(),
    GaussianNB(),
    svm.SVC(kernel = 'linear'),
    svm.SVC(kernel = 'sigmoid')
    
    ]
models = zip(names, Classifiers)

# Train each model on our dataset and look at the accuracy

for name, model in models:
    
    model.fit(X_train,y_train)
    train_acc = accuracy_score(y_train,model.predict(X_train))
    val_acc = accuracy_score(y_test,model.predict(X_test))
    
    msg = "{0}:\nTraining Accuracy : {1} Validation Accuracy : {2}\n\n".format(name, train_acc, val_acc)
    print(msg)



Decision Tree:
Training Accuracy : 0.4644906743185079 Validation Accuracy : 0.4376569037656904


Random Forest:
Training Accuracy : 0.47560975609756095 Validation Accuracy : 0.4535564853556485


Neural Net:
Training Accuracy : 0.41212338593974174 Validation Accuracy : 0.40460251046025103


AddaBoost:
Training Accuracy : 0.4266499282639885 Validation Accuracy : 0.41380753138075316


Naive Bayes:
Training Accuracy : 0.37392395982783355 Validation Accuracy : 0.3686192468619247


SVM Linear:
Training Accuracy : 0.398493543758967 Validation Accuracy : 0.39790794979079497


SVM Sigmoid:
Training Accuracy : 0.23637015781922524 Validation Accuracy : 0.24476987447698745




#GNN

In [ ]:
print('features: ', features.shape)
print('adjacency: ', adj.shape)
print('labels: ', labels.shape)

features:  (1, 7966, 18)
adjacency:  (1, 7966, 7966)
labels:  (7966, 1)


In [ ]:
# import packages

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
torch.set_printoptions(sci_mode=False)
import time

In [ ]:
# set initial model config
cuda = torch.cuda.is_available()
weight_decay = 1e-8
seed = 635
hidden = 16 
epochs = 10000
print('cuda: ', cuda)

np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

cuda:  True


In [ ]:
def normalize(adj):

    adj = torch.FloatTensor(adj)
    adj_id = torch.FloatTensor(torch.eye(adj.shape[1]))
    adj_id = adj_id.reshape((1, adj.shape[1], adj.shape[1]))
    adj_id = adj_id.repeat(adj.shape[0], 1, 1)
    adj = adj + adj_id
    rowsum = torch.FloatTensor(adj.sum(2))
    degree_mat_inv_sqrt = torch.diag_embed(torch.float_power(rowsum,-0.5), dim1=-2, dim2=-1).float()
    adj_norm = torch.bmm(torch.transpose(torch.bmm(adj,degree_mat_inv_sqrt),1,2),degree_mat_inv_sqrt)

    return adj_norm


In [ ]:
def doublerelu(x):
    return torch.clamp(x, 0, 1)

In [ ]:
class GNN1Layer(Module):

    def __init__(self, batch_size, in_features, out_features, first):
        super(GNN1Layer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.batch_size = batch_size
        
        # Initialse W1 = 1, W2 = 0 as pytorch learnable weights (parameters) that have require_grad = True which is
        # required for calculating gradients while backpropogating using gradient descent
        weight1_eye = torch.FloatTensor(torch.eye(in_features, out_features))
        weight1_eye = weight1_eye.reshape((1, in_features, out_features))
        weight1_eye = weight1_eye.repeat(batch_size, 1, 1)
        self.weight1 = Parameter(weight1_eye)
        if not first:
            self.weight2 = Parameter(torch.zeros(batch_size, in_features, out_features))
        else:
            self.weight2 = Parameter(torch.empty(batch_size, in_features, out_features))
            nn.init.kaiming_normal_(self.weight2, mode='fan_out')

    def forward(self, input, adj):
        # first term H*W1
        v1 = torch.bmm(input, self.weight1)
        # second term adj_norm*H*W2
        v2 = torch.bmm(torch.bmm(adj, input), self.weight2)
        # adding the two terms
        output = v1 + v2

        return output

In [ ]:
class GNN1(nn.Module):

    def __init__(self, batch_size, nfeat, ndim, hidden, first):
        super(GNN1, self).__init__()

        self.gc1 = GNN1Layer(batch_size, nfeat, hidden, first)
        self.gc2 = GNN1Layer(batch_size, hidden, ndim, first)

    def forward(self, x, adj):

        # Applying activation function sigma (doublerelu) on the layer propogation
        x = doublerelu(self.gc1(x, adj))
        x = doublerelu(self.gc2(x, adj))
        x = x/x.sum(axis=2).unsqueeze(2) #normalize st sum = 1
        
        return x

In [ ]:
def train(adj,features,labels,train_indices,val_indices,first=False):
    
    # calculate symmetric normalisation for layer propogation
    adj_norm = normalize(adj)
    
    labels = labels - 1


    
    # Convert from numpy to torch tensors
    adj = torch.FloatTensor(adj)
    adj_norm = torch.FloatTensor(adj_norm)
    features = torch.FloatTensor(features)
    labels = torch.FloatTensor(labels)
    
    # initialise the model
    model = GNN1(batch_size=adj.shape[0],
                nfeat=features.shape[-1],
                ndim=5,
                hidden=hidden,
                first=first)
    
    # Transfer the weights to GPU for training
    if cuda:
        model.cuda()
        features = features.cuda()
        adj = adj.cuda()
        adj_norm = adj_norm.cuda()
        labels = labels.cuda()
    
    # Train model
    t_total = time.time()

    # Using adam optimizers for backpropogation
    optimizer = optim.Adam(model.parameters(),
                           lr=lr, weight_decay=weight_decay)
    
    # loss function criteria is cross entropy loss
    criterion = nn.CrossEntropyLoss()
    
    # Train for the no of epochs

    for epoch in range(epochs):

        t = time.time()
        
        model.train()
        
        # Pytorch accumulates gradient after every operation on tensors (defined by the model architecture)
        # with require_grad = True. With each new epoch, we need to reset this gradient to 0 to calculate gradient
        # for this epoch.
        optimizer.zero_grad()

        # get the output from forward propogation of our model
        output = model(features, adj_norm)
        
        # Calculate Train accuracy
        train_output = output[:,train_indices,:]
        train_labels = labels[train_indices,:]
        train_accuracy = torch.sum(torch.argmax(train_output,axis=2)==train_labels.reshape(1,-1))/train_labels.shape[0]
        
        # Calculate the loss between our models training output and true label
        loss = criterion(output[0],labels.reshape(-1).long())



        # Calculate the gradients 
        loss.backward(retain_graph=True)

        # Update the weights
        optimizer.step()
        
        model.eval()
        
        # Calculate Validation accuracy
        with torch.no_grad():
            val_output = output[:,val_indices,:]
            val_labels = labels[val_indices,:]
            val_accuracy = torch.sum(torch.argmax(val_output,axis=2)==val_labels.reshape(1,-1))/val_labels.shape[0]

        # Print summary of training 
        if epoch == 0:
            best_loss = loss
            best_output = output
            best_acc = train_accuracy
            best_val_acc = val_accuracy
            best_val_output = val_output
        else:
            if loss < best_loss:
                best_loss = loss
                best_output = output
                best_acc = train_accuracy
                best_val_acc = val_accuracy
                best_val_output = val_output
        

        if epoch == 0 or (epoch+1) % 1000 == 0:
            print('Epoch: {:04d}'.format(epoch + 1),
                  'Train Accuracy: {:.4f}'.format(best_acc.item()),
                  'Validation Accuracy: {:.4f}'.format(best_val_acc.item()),
                  'Loss: {:.8f}'.format(best_loss.item()),
                  'time: {:.4f}s'.format(time.time() - t))
                  
            loss_list.append(loss.item())
            acc_list.append(train_accuracy.item())
            np.save('/content/drive/MyDrive/DL/Untitled folder/loss_list.npy', loss_list)
            np.save('/content/drive/MyDrive/DL/Untitled folder/acc_list.npy', acc_list)
            if not os.path.isdir(f'/content/drive/MyDrive/DL/checkpoint'):
              os.mkdir(f'/content/drive/MyDrive/DL/checkpoint')
              torch.save(f'./content/drive/MyDrive/DL/checkpoint/GNN.pth')

        
            
    print("Optimization Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
    
    return best_loss,best_output,loss_list,acc_list#,best_val_output

In [ ]:
# set Train %
train_percentage = .8
# set initial model config

cuda = torch.cuda.is_available()
weight_decay = 1e-8
epochs = 10000
seed = 635
hidden = 16
lr = 0.001
epochs = 10000

loss_list=[]
acc_list=[]

# Train set
number_of_rows = features[0].shape[0]
train_indices = np.random.choice(number_of_rows, size=int(train_percentage*number_of_rows), replace=False)
val_indices = np.setdiff1d(np.arange(adj.shape[1]),train_indices)

# Start Train
prev_loss, op,loss_list,acc_list = train(adj,features,labels,train_indices,val_indices,True)

# Keep training recurrently until the loss stops decreasing
loss, op,loss_list,acc_list  = train(adj,op.cpu().detach().numpy(),labels,train_indices,val_indices)
while loss < prev_loss :
    prev_loss = loss
    loss, op,loss_list,acc_list  = train(adj,op.cpu().detach().numpy(),labels,train_indices,val_indices)



Epoch: 0001 Train Accuracy: 0.1737 Validation Accuracy: 0.1945 Loss: 1.62723470 time: 0.0096s
Epoch: 1000 Train Accuracy: 0.2203 Validation Accuracy: 0.2240 Loss: 1.51188481 time: 0.0063s
Epoch: 2000 Train Accuracy: 0.2203 Validation Accuracy: 0.2240 Loss: 1.49633849 time: 0.0064s
Epoch: 3000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0078s
Epoch: 4000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0065s
Epoch: 5000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0066s
Epoch: 6000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0064s
Epoch: 7000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0064s
Epoch: 8000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0063s
Epoch: 9000 Train Accuracy: 0.2250 Validation Accuracy: 0.2277 Loss: 1.49113798 time: 0.0062s
Epoch: 10000 Train Accuracy: 0.2250 Validation Accuracy: 0.2